In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk 
import string
import re
import codecs
import json
#%matplotlib inline
#pd.set_option('display.max_colwidth', 100)

import hdbscan
import nltk
import numpy as np
import pandas as pd
import random
import re
import spacy
import textacy
import csv
import sklearn

from gmplot import gmplot
from mapsplotlib import mapsplot as mplt
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag_sents
from pymprog import *
from sklearn.cluster import DBSCAN
from sklearn.neighbors import DistanceMetric
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd

import datetime
from time import strptime

from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from sklearn.cluster import KMeans


In [4]:
fname = "relevant_json.json"
list_tweets = []
count = 0 
sum = 1000
with open(fname,encoding="utf8") as f:
    for line in f:
        try:
            tweet = json.loads(line)
        except:
            pass
        list_tweets.append(tweet)
#         count+=1
#         if count==sum:
#             print(count)
#             sum*=2
# list_tweets[0].keys()

df_relevant = pd.DataFrame(data=[tweet["full_text"] for tweet in list_tweets], columns=['tweets'])
df_relevant['created_at'] = np.array([tweet["created_at"] for tweet in list_tweets])
df_relevant['id'] = np.array([int(tweet["id_str"]) for tweet in list_tweets])
df_relevant['rt'] = np.array([tweet["retweeted"] for tweet in list_tweets])
df_relevant['lang'] = np.array([tweet["lang"] for tweet in list_tweets])
df_relevant['Predicted_Event'] = -1

# df_relevant.shape[0]

fname = "relevant_tweets.tsv"
event_id = pd.read_csv(fname, sep='\t',header=None,names=['Event','id'])

# event_id.shape[0]

df_relevant = df_relevant.merge(event_id, left_on='id', right_on='id')
df_relevant.head(1)

,tweets,created_at,id,rt,lang,Predicted_Event,Event
0,News: Chinese author Mo Yan wins Nobel http://...,Thu Oct 11 11:08:32 +0000 2012,256350567369175040,False,en,-1,0


In [5]:
colnames=['tweets', 'created_at', 'id','rt','lang'] 
df_irrelevant = pd.read_csv('output_1.csv', names=colnames, header=None)
df_irrelevant['Predicted_Event'] = -1
df_irrelevant['Event'] = -1
print(df_irrelevant.shape[0])
df_irrelevant.head(1)

413986


,tweets,created_at,id,rt,lang,Predicted_Event,Event
0,I love ms.sivney class (Sports &amp; Entertaim...,Wed Oct 31 16:48:39 +0000 2012,263683917889040400,False,en,-1,-1


In [6]:
# fname = "irrelevant_tweets"
# irrelevant_list_tweets = []
# count = 0 
# sum = 1000
# with open(fname,encoding="utf8") as f:
#     for line in f:
#         try:
#             tweet = json.loads(line)
#         except:
#             pass
#         irrelevant_list_tweets.append(tweet)
# #         count+=1
# #         if count==sum:
# #             print(count)
# #             sum*=2
# # irrelevant_list_tweets[0].keys()

# df_irrelevant = pd.DataFrame(data=[tweet["full_text"] for tweet in irrelevant_list_tweets], columns=['tweets'])
# df_irrelevant['created_at'] = np.array([tweet["created_at"] for tweet in irrelevant_list_tweets])
# df_irrelevant['id'] = np.array([tweet["id"] for tweet in irrelevant_list_tweets])
# df_irrelevant['rt'] = np.array([tweet["retweeted"] for tweet in irrelevant_list_tweets])
# df_irrelevant['lang'] = np.array([tweet["lang"] for tweet in irrelevant_list_tweets])
# df_irrelevant['Predicted_Event'] = -1
# df_irrelevant['Event'] = -1

# df_irrelevant.shape[0]

In [7]:
frames = [df_relevant,df_irrelevant]
df = pd.concat(frames)

In [8]:
# frames = [df_relevant,df_irrelevant]
# df = pd.concat(frames)
# # df

# import matplotlib.pyplot as plt
# # plots number of tweets in each language
# counts = df['lang'].value_counts()
# plt.bar(counts.index, counts, align='center', alpha=1)
# plt.xlabel('Tweet Language')
# plt.ylabel('# of Tweets')
# plt.title('Tweets 2012')

In [9]:
print("Old df shape:",df.shape[0])
df = df.loc[df["lang"]=='en']
print("New df shape:",df.shape[0])

Old df shape: 520003
New df shape: 507519


In [10]:
df = df.reset_index(drop=True)

def create_date(text):
    temp = text.split(" ")
    x = str(datetime.datetime(int(temp[5]),int(strptime(temp[1],'%b').tm_mon),int(temp[2]))).split()[0]
    return x

df['date'] = df['created_at'].apply(lambda x: create_date(x))
df['time'] =  np.array([tweet.split()[3] for tweet in df["created_at"]])
df['Datetime']= pd.to_datetime(df['date'].apply(str)+' '+df['time'].apply(lambda x: x.split(':')[0]) + df['time'].apply(lambda x: x.split(':')[1]))
df['DateHour'] = pd.to_datetime(df['date'].apply(str)+' '+df['time'].apply(lambda x: x.split(':')[0])+':00')
df['DateMinutes'] = pd.to_datetime(df['date'].apply(str)+' '+df['time'].apply(lambda x: x.split(':')[0])+':'+df["time"].apply(lambda x: x.split(":")[1][0]+'0'))

tableFlag=[]
data_sorted = df.sort_values(by=['date'],inplace=False)
data_sorted = data_sorted.reset_index(drop=True)

list_date = []
for temp in data_sorted["DateHour"]:
    list_date.append(temp)

list_date = list(dict.fromkeys(list_date))
list_date = sorted(list_date)
print(len(list_date))
# list_date
# timeseries_y = []
# for i in range(0,len(list_date)):
#     timeseries_y.append(df.loc[df["DateHour"]==list_date[i]].shape[0])
# plt.plot(list_date,timeseries_y)

673


In [11]:
# for current_date in list_date:
#     print(df.loc[df["DateHour"]==current_date].shape[0])

In [12]:
df_relevant['date'] = df_relevant['created_at'].apply(lambda x: create_date(x))
df_relevant['time'] =  np.array([tweet.split()[3] for tweet in df_relevant["created_at"]])
df_relevant['DateHour'] = pd.to_datetime(df_relevant['date'].apply(str)+' '+df_relevant['time'].apply(lambda x: x.split(':')[0])+':00')
for current_date in list_date:
    print(df_relevant.loc[df_relevant["DateHour"]==current_date].shape[0])

1112
1288
697
43
46
39
57
41
29
42
95
92
141
100
79
76
100
99
137
84
92
53
60
58
40
21
16
107
16
65
32
51
37
65
120
195
96
82
63
123
153
127
314
109
59
66
44
441
169
5251
1426
89
79
20
30
102
104
823
96
91
78
66
70
63
48
62
88
200
333
151
59
52
141
94
56
78
718
19
31
154
33
23
25
65
52
43
92
67
58
67
41
47
36
70
94
91
191
53
48
160
200
503
14
38
50
38
43
84
59
60
57
48
496
1455
1130
172
90
28
31
93
57
39
36
256
37
30
57
75
68
65
74
115
52
68
38
63
60
49
37
43
1022
28
61
132
239
56
131
906
35
20
15
51
81
33
129
57
102
132
250
87
217
106
400
207
613
270
139
759
227
9206
2846
198
42
46
40
62
43
60
71
45
184
365
61
81
66
120
61
80
102
66
112
42
25
25
23
35
19
21
12
22
19
18
44
99
106
68
72
60
62
106
81
62
127
262
51
245
27
15
28
26
36
12
21
40
28
49
30
64
54
77
61
115
634
174
506
140
363
41
35
18
31
21
7
16
15
14
24
28
73
57
99
39
822
533
491
192
75
32
106
350
122
77
12
109
28
13
15
19
9
16
14
22
39
83
34
40
142
560
193
81
79
86
175
52
622
134
76
313
60
29
19
23
24
23
41
23
31
52
69
192
10

In [13]:
# list_date

In [14]:
def remove_patterns(input_txt,mentions,hashtags,RT):
    m = re.findall(mentions, input_txt)
    h = re.findall(hashtags,input_txt)
    r = re.findall(RT,input_txt)
    u = re.findall(r"http\S+",input_txt)
    mention_list = []
    hashtag_list = []
    url_list = []
    rt1 = 0
    
    
    for i in u:
        url_list.append(i)
        #input_txt = re.sub(i,'',input_txt)
    input_txt = re.sub(r"http\S+",'',input_txt)
    for i in m:
        mention_list.append(i)
        input_txt = re.sub(i, '', input_txt)
    for i in h:
        hashtag_list.append(i)
        input_txt = re.sub(i,'',input_txt)
    if r!=0 :
        rt1 = 1
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    #Some json
    x = {"input_text" : input_txt,"user_mentions":mention_list,"hashtags":hashtag_list,"is_RT":rt1,"is_URL":url_list}
    #y = json.dumps(x)
    return x

def list_lower(temp):
    list_lower =[]
    for text in temp:
        list_lower.append(text.lower())
    return list_lower

temp = np.vectorize(remove_patterns)(df['tweets'],"@[\w]*","#[\w]*","RT")

#print(df["Orig_Tweet"][12])
df["text"] = np.array([text["input_text"] for text in temp])
#df2["hashtag"] = np.array([text["hashtags"] for text in temp])
df["hashtag"] = np.array([list_lower(text["hashtags"]) for text in temp])
df["user_mentions"] = np.array([text["user_mentions"] for text in temp]) 
df["RT"] = np.array([text["is_RT"] for text in temp])
df["URL"] = np.array([text["is_URL"] for text in temp])

In [15]:
def remove_punct(text):
    temp = string.punctuation+"…"
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def tokenization(text):
    text = re.split('\W+', text)
    #text = nltk.word_tokenize(text)
    temp=[]
    for i in text:
        if i!='':
            temp.append(i)
    return temp

df['Tweet_punct'] = df['text'].apply(lambda x: remove_punct(x))
df['Tweet_tokenized'] = df['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
# df.head()

In [16]:
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

df['Tweet_nonstop'] = df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))

In [17]:
# Stemming is the process of breaking a word down into its root.
stemmer = SnowballStemmer('english')
stops = set(stopwords.words("english"))

def stemming_to_sent(words):
    stemmed_words = [stemmer.stem(w) for w in words]
    return(" ".join(stemmed_words))
def stemming_to_words(words):
    stemmed_words = [stemmer.stem(w) for w in words]
    return stemmed_words

df["Stem_sent"] = np.array([stemming_to_sent(temp) for temp in df["Tweet_nonstop"]])
df["Stem_words"] = np.array([stemming_to_words(temp) for temp in df["Tweet_nonstop"]])
df["No"] = np.array([i for i in range(0,df.shape[0])])

In [18]:
def new_clean_data(i,df):
    sent = ""
    for temp in df["Stem_words"].get(i):
        sent+= temp + " "
    for temp in df["hashtag"].get(i):
        sent+= temp + " "
    for temp in df["user_mentions"].get(i):
        sent+= temp+ " "
    for temp in df["URL"].get(i):
        sent+= temp + " "
    return sent

df["clean_data"] = np.array([new_clean_data(i,df) for i in range(df.shape[0])])

In [19]:
# Function to convert   
def listToString(s):
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

def lists_to_sentence(tokens,hashtag):
    t = tokens.copy()
    h = hashtag.copy()
    for temp in h:
        t.append(temp)
    return listToString(t)

# total_df["total_sentence"] = np.array([lists_to_sentence(row["Stem_words"],row["hashtag"]) for index,row in total_df.iterrows()])
# df

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
def getTfIdf(clean_train,windows_corpus,ngram1,ngram2,mindf,maxdf):
    if (mindf==0):
        minimum = 1
    else:
        minimum = max(int(windows_corpus * 0.0025), 10)
    if (maxdf==0):
        maximum = 1.0
    else:
        maximum = 0.8  
        
    tfidf_vectorizer = TfidfVectorizer(
                                       min_df=minimum,
                                       max_df=maximum,
                                       max_features=2000000,
                                       stop_words='english',
                                       use_idf=True,
                                       ngram_range=(ngram1,ngram2)
    )
    # Tf-idf-weighted term-document sparse matrix
    try :
        tfidf_train_data_features = tfidf_vectorizer.fit_transform(clean_train)
    except:
#         print("Hello")
        tfidf_vectorizer = TfidfVectorizer(max_features=2000000,stop_words='english',use_idf=True,ngram_range=(1,3))
        tfidf_train_data_features = tfidf_vectorizer.fit_transform(clean_train)
        minimum = 1
        maximum = 1.0
    finally:
        list_sum = []
        for i in range(0,tfidf_train_data_features.shape[0]):
            list_sum.append(tfidf_train_data_features[i].count_nonzero())
                
        return {'tfidf_train_data_features':tfidf_train_data_features,'minimum':minimum,'maximum':maximum,'sum':list_sum}

In [21]:
# function to get unique values 
def getNumClusters(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return len(unique_list)

# take second element for sort
def takeSecond(elem):
    return elem[1]

In [150]:
from sklearn.cluster import AffinityPropagation
def getAffinity(tweets,tfidf):
    clustering = AffinityPropagation().fit(tfidf.toarray())
    
    clusters = clustering.labels_.tolist()
    temp = clusters.copy()
    temp.remove
    
    addedCluster = tweets.copy()
    addedCluster['Cluster'] = clusters
    num_clusters = getNumClusters(clusters)

    pd.options.display.max_colwidth = 100

    cls = []
    #num_clusters = 55
    max_num_cluster = 0

    for i in range(0, num_clusters):
        tweetsInCluster = addedCluster[addedCluster['Cluster'] == i]
        if(tweetsInCluster.shape[0] > 0):
            cls.append((tweetsInCluster,tweetsInCluster.shape[0]))
            
#     cls.sort(key=takeSecond,reverse=True)
    clusters = []
    for i in range(len(cls)):
        clusters.append(cls[i][0])
    
    return {"clusters":clusters,"labels_pr":clustering.labels_.tolist()}
    

In [148]:
from sklearn.cluster import KMeans
def getKMeans(tweets,tfidf):
    clustering = KMeans(n_clusters=5,n_jobs=-1).fit(tfidf.toarray())
    
    clusters = clustering.labels_.tolist()
    temp = clusters.copy()
    temp.remove
    
    addedCluster = tweets.copy()
    addedCluster['Cluster'] = clusters
    num_clusters = getNumClusters(clusters)

    pd.options.display.max_colwidth = 100

    cls = []
    #num_clusters = 55
    max_num_cluster = 0

    for i in range(0, num_clusters):
        tweetsInCluster = addedCluster[addedCluster['Cluster'] == i]
        if(tweetsInCluster.shape[0] > 0):
            cls.append((tweetsInCluster,tweetsInCluster.shape[0]))
            
#     cls.sort(key=takeSecond,reverse=True)
    clusters = []
    for i in range(len(cls)):
        clusters.append(cls[i][0])
    
    return {"clusters":clusters,"labels_pr":clustering.labels_.tolist()}
    

In [147]:
from sklearn.cluster import AgglomerativeClustering
def getHierarchical(tweets,tfidf):
    clustering = AgglomerativeClustering(n_clusters=5).fit(tfidf.toarray())
    
    clusters = clustering.labels_.tolist()
    temp = clusters.copy()
    temp.remove
    
    addedCluster = tweets.copy()
    addedCluster['Cluster'] = clusters
    num_clusters = getNumClusters(clusters)

    pd.options.display.max_colwidth = 100

    cls = []
    #num_clusters = 55
    max_num_cluster = 0

    for i in range(0, num_clusters):
        tweetsInCluster = addedCluster[addedCluster['Cluster'] == i]
        if(tweetsInCluster.shape[0] > 0):
            cls.append((tweetsInCluster,tweetsInCluster.shape[0]))
            
#     cls.sort(key=takeSecond,reverse=True)
    clusters = []
    for i in range(len(cls)):
        clusters.append(cls[i][0])
    
    return {"clusters":clusters,"labels_pr":clustering.labels_.tolist()}
    

In [149]:
def getHDBSCAN(tweets,tfidf):
    clustering = hdbscan.HDBSCAN(min_cluster_size=5,metric='cosine').fit(tfidf)
    
    clusters = clustering.labels_.tolist()
    temp = clusters.copy()
    temp.remove
    
    addedCluster = tweets.copy()
    addedCluster['Cluster'] = clusters
    num_clusters = getNumClusters(clusters)

    pd.options.display.max_colwidth = 100

    cls = []
    #num_clusters = 55
    max_num_cluster = 0

    for i in range(0, num_clusters):
        tweetsInCluster = addedCluster[addedCluster['Cluster'] == i]
        if(tweetsInCluster.shape[0] > 0):
            cls.append((tweetsInCluster,tweetsInCluster.shape[0]))
            
#     cls.sort(key=takeSecond,reverse=True)
    clusters = []
    for i in range(len(cls)):
        clusters.append(cls[i][0])
        
    #Noise Data
    noise_data = addedCluster[addedCluster['Cluster']==-1]
#     noise_data = addedCluster[addedCluster['Cluster'] == -1]
#     print("Size:",noise_data.shape[0])
#     print(noise_data["tweets"],noise_data["Event"])
    
    return {"clusters":clusters,"noise_data":noise_data,"labels_pr":clustering.labels_.tolist()}

In [146]:
def getDBSCAN(tweets,tfidf,e):
    clustering = DBSCAN(eps=e, min_samples=5,n_jobs=-1,metric='cosine').fit(tfidf)
    
    clusters = clustering.labels_.tolist()
    temp = clusters.copy()
    temp.remove
    
    addedCluster = tweets.copy()
    addedCluster['Cluster'] = clusters
    num_clusters = getNumClusters(clusters)

    pd.options.display.max_colwidth = 100

    cls = []
    #num_clusters = 55
    max_num_cluster = 0

    for i in range(0, num_clusters):
        tweetsInCluster = addedCluster[addedCluster['Cluster'] == i]
        if(tweetsInCluster.shape[0] > 0):
            cls.append((tweetsInCluster,tweetsInCluster.shape[0]))
            
#     cls.sort(key=takeSecond,reverse=True)
    clusters = []
    for i in range(len(cls)):
        clusters.append(cls[i][0])
        
    #Noise Data
    noise_data = addedCluster[addedCluster['Cluster']==-1]
#     noise_data = addedCluster[addedCluster['Cluster'] == -1]
#     print("Size:",noise_data.shape[0])
#     print(noise_data["tweets"],noise_data["Event"])
    
    return {"clusters":clusters,"noise_data":noise_data,"labels_pr":clustering.labels_.tolist()}

In [185]:
def getLabel(list_event):
    temp = list_event.value_counts()
    return temp.keys()[0]

from sklearn.preprocessing import StandardScaler
import statistics
def create_clusters(df,ngram,epsilon,mindf,maxdf,method_cl):  
    list_y_fscore = []
    list_pr_fscore = []
    adjusted_mutual_info = []
    adjusted_rand_score = []
    completeness_score = []
    fowlkes_mallows_score = []
    homogeneity_score = []
    mutual_info_score = []
    normalized_mutual_info_score = []
    v_measure_score = []
    silhouette = []
    calinski_harabasz = []
    davies_bouldin = []
    contingency = []
    count = 0
    count_labels = 0
    for current_date in list_date:
        try:
            list_y = []
            list_pr = []
            
            dfchange = df.loc[df['DateHour']==current_date]
            tweets = dfchange.sort_values(by=['time'])
            tweetsContent = tweets.copy()["clean_data"]
            tfidf_object = getTfIdf(tweetsContent,dfchange.shape[0],ngram,3,mindf,maxdf)
            tfidf = tfidf_object['tfidf_train_data_features']
            minimum = tfidf_object['minimum']
            maximum = tfidf_object['maximum']
            list_sum = tfidf_object['sum']
            tweets['sum'] = list_sum
            noise_tweets = tweets.loc[tweets['sum']==0]
            
#             print("Event",count,":")
            count+=1
#             print("Cluster_noise: ",noise_tweets.shape[0],"False Negative:",noise_tweets.loc[noise_tweets["Event"]!=-1].shape[0])
            for index,row in noise_tweets.iterrows():
                list_y_fscore.append(row["Event"])
                list_pr_fscore.append(-1)
#                 print(row["Event"]," ",end='')
#             print()
            
            train_tweets = tweets.loc[tweets['sum']!=0]
#             print("Tweets: ",train_tweets.shape[0])
            trainContent = train_tweets.copy()["clean_data"]
            
#             import pdb; pdb.set_trace()
            headline_vectorizer = CountVectorizer(binary=True, min_df=1,ngram_range=(1,1))
            tfidf = headline_vectorizer.fit_transform(trainContent)
            
            if (method_cl == 'dbscan'):
                x_object = getDBSCAN(train_tweets,tfidf,epsilon)
            elif (method_cl == 'hdbscan'):
                x_object = getHDBSCAN(train_tweets,tfidf)
            elif (method_cl == 'hierarchical'):
                x_object = getHierarchical(train_tweets,tfidf)
            elif (method_cl == 'kmeans'):
                x_object = getKMeans(train_tweets,tfidf)
            else:
                x_object = getAffinity(train_tweets,tfidf)

            candidate_cluster = x_object["clusters"]
            if (method_cl=='dbscan' or method_cl=='hdbscan'):
                candidate_noise = x_object["noise_data"]
                for index,row in candidate_noise.iterrows():
                    list_y.append(row["Event"])
                    list_pr.append(-1)
                    list_y_fscore.append(row["Event"])
                    list_pr_fscore.append(-1)
            candidate_labels= x_object["labels_pr"]
            
            
            for current_cluster in candidate_cluster:
#                 print("Cluster Label:",count_labels,current_cluster.shape[0])
                pr_event = getLabel(current_cluster["Event"])
                for index,row in current_cluster.iterrows():
                    list_y.append(row["Event"])
                    list_pr.append(count_labels)
                    list_y_fscore.append(row["Event"])
                    list_pr_fscore.append(pr_event)
#                     print(row["Event"]," ",end='')
#                 print()
                count_labels+=1

            
            from sklearn import metrics
            adjusted_mutual_info.append(metrics.adjusted_mutual_info_score(list_y,list_pr,average_method='arithmetic'))
            adjusted_rand_score.append(metrics.adjusted_rand_score(list_y,list_pr))
            completeness_score.append(metrics.completeness_score(list_y,list_pr))
            fowlkes_mallows_score.append(metrics.fowlkes_mallows_score(list_y,list_pr,sparse=False))
            homogeneity_score.append(metrics.homogeneity_score(list_y,list_pr))
            mutual_info_score.append(metrics.mutual_info_score(list_y,list_pr,contingency=None))
            normalized_mutual_info_score.append(metrics.normalized_mutual_info_score(list_y,list_pr,average_method='arithmetic'))
            v_measure_score.append(metrics.v_measure_score(list_y,list_pr))
        except Exception as e:
            print(e)
            pass
    from sklearn import metrics
    f1 = metrics.f1_score(list_y_fscore,list_pr_fscore,average='micro')
    clustering_name = ''
    num_grams = '('+str(ngram)+',3)'
    details = {'clustering_name':method_cl,'num_grams':num_grams,'epsilon':epsilon,
                   'min_df':minimum,'max_df':maximum}
    
    try:
        clustering_object =  {'f1_score':f1,'adjusted_mutual_info_score':statistics.mean(adjusted_mutual_info),'adjusted_rand_score':statistics.mean(adjusted_rand_score),
                                'completeness_score':statistics.mean(completeness_score),'fowlkes_mallows_score':statistics.mean(fowlkes_mallows_score),
                                'homogeneity_score':statistics.mean(homogeneity_score),'mutual_info_score':statistics.mean(mutual_info_score),
                                'normalized_mutual_info_score':statistics.mean(normalized_mutual_info_score),
                                 'v_measure_score':statistics.mean(v_measure_score)}

        total_result = {'parameters':details,'clustering_object':clustering_object}
        return json.dumps(total_result)
    except Exception as e:
        print(e)
        return None
    return None

In [122]:
df = df.sort_values(by=['Datetime'])
df["Timer"] = 0

In [186]:
result1 = create_clusters(df,2,0.5,1,0,'dbscan')
print(result1)

{"parameters": {"clustering_name": "dbscan", "num_grams": "(2,3)", "epsilon": 0.5, "min_df": 1, "max_df": 1.0}, "clustering_object": {"f1_score": 0.9047641566128559, "adjusted_mutual_info_score": 0.38504509425685907, "adjusted_rand_score": 0.3962618185414322, "completeness_score": 0.5696409671007676, "fowlkes_mallows_score": 0.8629620668201434, "homogeneity_score": 0.4884512363825032, "mutual_info_score": 0.2282049673838112, "normalized_mutual_info_score": 0.3993838179427191, "v_measure_score": 0.3993838179427191}}


In [187]:
result2 = create_clusters(df,2,0.5,1,0,'hdbscan')
print(result2)

Invalid shape in axis 0: 0.
{"parameters": {"clustering_name": "hdbscan", "num_grams": "(2,3)", "epsilon": 0.5, "min_df": 1, "max_df": 1.0}, "clustering_object": {"f1_score": 0.8782072754069806, "adjusted_mutual_info_score": 0.35085944711469114, "adjusted_rand_score": 0.3220718384688426, "completeness_score": 0.4623246041285216, "fowlkes_mallows_score": 0.7949029320872439, "homogeneity_score": 0.44857772239835436, "mutual_info_score": 0.21786803114652611, "normalized_mutual_info_score": 0.3688643818772194, "v_measure_score": 0.3688643818772194}}


In [188]:
result3 = create_clusters(df,2,0.5,1,0,'hierarchical')
print(result3)

Found array with 1 sample(s) (shape=(1, 12)) while a minimum of 2 is required by AgglomerativeClustering.
{"parameters": {"clustering_name": "hierarchical", "num_grams": "(2,3)", "epsilon": 0.5, "min_df": 1, "max_df": 1.0}, "clustering_object": {"f1_score": 0.9178176931655626, "adjusted_mutual_info_score": 0.2482712851207647, "adjusted_rand_score": 0.1926614856575077, "completeness_score": 0.22975990317855102, "fowlkes_mallows_score": 0.6167201867867069, "homogeneity_score": 0.6138740163490718, "mutual_info_score": 0.29127423447440287, "normalized_mutual_info_score": 0.2863451597557061, "v_measure_score": 0.2863451597557061}}


In [189]:
result4 = create_clusters(df,2,0.5,1,0,'kmeans')
print(result4)

C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)
C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number o

n_samples=1 should be >= n_clusters=5
{"parameters": {"clustering_name": "kmeans", "num_grams": "(2,3)", "epsilon": 0.5, "min_df": 1, "max_df": 1.0}, "clustering_object": {"f1_score": 0.9153921634306567, "adjusted_mutual_info_score": 0.22017460250911838, "adjusted_rand_score": 0.1814955453427232, "completeness_score": 0.2099725178252495, "fowlkes_mallows_score": 0.6224699504369879, "homogeneity_score": 0.5592590027338353, "mutual_info_score": 0.26700898064216777, "normalized_mutual_info_score": 0.2579999150978515, "v_measure_score": 0.2579999150978515}}


In [190]:
result5 = create_clusters(df,2,0.5,1,0,'affinity')
print(result5)

C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\affinity_propagation_.py:125: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn("All samples have mutually equal similarities. "
C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\affinity_propagation_.py:125: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn("All samples have mutually equal similarities. "


{"parameters": {"clustering_name": "affinity", "num_grams": "(2,3)", "epsilon": 0.5, "min_df": 1, "max_df": 1.0}, "clustering_object": {"f1_score": 0.9226511716802721, "adjusted_mutual_info_score": 0.20247489190319304, "adjusted_rand_score": 0.14692637309718914, "completeness_score": 0.2119806762187904, "fowlkes_mallows_score": 0.5357817358354268, "homogeneity_score": 0.6829826882263953, "mutual_info_score": 0.3054976226978315, "normalized_mutual_info_score": 0.25174537750889425, "v_measure_score": 0.25174537750889425}}


In [174]:
from sklearn.preprocessing import StandardScaler
import statistics
def create_clusters_show(df,ngram,epsilon,mindf,maxdf,method_cl):  
    count = 0
    count_labels = 0
    for current_date in list_date[0:100]:
        try:
            list_y = []
            list_pr = []
            
            dfchange = df.loc[df['DateHour']==current_date]
            tweets = dfchange.sort_values(by=['time'])
            tweetsContent = tweets.copy()["clean_data"]
            tfidf_object = getTfIdf(tweetsContent,dfchange.shape[0],ngram,3,mindf,maxdf)
            tfidf = tfidf_object['tfidf_train_data_features']
            minimum = tfidf_object['minimum']
            maximum = tfidf_object['maximum']
            list_sum = tfidf_object['sum']
            tweets['sum'] = list_sum
            noise_tweets = tweets.loc[tweets['sum']==0]
            
            print("Event",count,":")
            count+=1
            print("Cluster_noise: ",noise_tweets.shape[0],"False Negative:",noise_tweets.loc[noise_tweets["Event"]!=-1].shape[0])
            
            train_tweets = tweets.loc[tweets['sum']!=0]
            print("Tweets: ",train_tweets.shape[0])
            trainContent = train_tweets.copy()["clean_data"]
            
#             import pdb; pdb.set_trace()
            headline_vectorizer = CountVectorizer(binary=True, min_df=1,ngram_range=(1,1))
            tfidf = headline_vectorizer.fit_transform(trainContent)
            
            if (method_cl == 'dbscan'):
                x_object = getDBSCAN(train_tweets,tfidf,epsilon)
            elif (method_cl == 'hdbscan'):
                x_object = getHDBSCAN(train_tweets,tfidf)
            elif (method_cl == 'hierarchical'):
                x_object = getHierarchical(train_tweets,tfidf)
            elif (method_cl == 'kmeans'):
                x_object = getKMeans(train_tweets,tfidf)
            else:
                x_object = getAffinity(train_tweets,tfidf)

            candidate_cluster = x_object["clusters"]
            if (method_cl=='dbscan' or method_cl=='hdbscan'):
                candidate_noise = x_object["noise_data"]

            candidate_labels= x_object["labels_pr"]
            
            
            for current_cluster in candidate_cluster:
                print("Cluster Label:",count_labels,current_cluster.shape[0])
                pr_event = getLabel(current_cluster["Event"])
                for index,row in current_cluster.iterrows():
                    print(row["Event"]," ",end='')
                print()
                count_labels+=1
                
        except Exception as e:
            print(e)
            pass

In [176]:
dbscan_show = create_clusters_show(df,2,0.5,1,0,'dbscan')

Event 0 :
Cluster_noise:  1442 False Negative: 362
Tweets:  802
Cluster Label: 0 278
394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  501  501  394  394  394  394  394  394  394  394  394  394  394  394  394  394  -1  394  394  394  394  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  394

Event 2 :
Cluster_noise:  1036 False Negative: 126
Tweets:  548
Cluster Label: 13 474
18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  -1  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  -1  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  -1  18  18  18  18

Event 33 :
Cluster_noise:  285 False Negative: 35
Tweets:  29
Cluster Label: 62 13
422  422  422  422  422  422  422  422  422  422  422  422  422  
Cluster Label: 63 6
239  239  426  426  239  239  
Event 34 :
Cluster_noise:  376 False Negative: 34
Tweets:  84
Cluster Label: 64 50
106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  
Cluster Label: 65 18
419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  
Cluster Label: 66 5
426  239  426  239  239  
Event 35 :
Cluster_noise:  484 False Negative: 47
Tweets:  159
Cluster Label: 67 125
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 

  1  1  1  1  1  1  -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  -1  1  1  1  1  
Cluster Label: 102 8
1  1  1  1  1  1  1  1  
Cluster Label: 103 15
1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  
Cluster Label: 104 10
1  1  1  1  1  1  1  1  1  1  
Cluster Label: 105 10
85  85  85  85  85  85  85  85  85  85  
Cluster Label: 106 6
85  1  1  1  85  1  
Cluster Label: 107 6
1  1  85  1  85  1  
Cluster Label: 108 6
1  85  1  

Event 59 :
Cluster_noise:  460 False Negative: 51
Tweets:  41
Cluster Label: 133 23
2  2  2  2  2  2  2  2  2  0  2  -1  2  0  2  2  2  2  2  2  2  2  2  
Cluster Label: 134 15
478  478  478  478  478  478  478  478  478  478  478  478  478  478  478  
Event 60 :
Cluster_noise:  505 False Negative: 58
Tweets:  30
Cluster Label: 135 8
-1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 136 19
2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  
Event 61 :
Cluster_noise:  602 False Negative: 53
Tweets:  13
Cluster Label: 137 12
2  2  2  2  2  2  2  2  2  2  2  2  
Event 62 :
Cluster_noise:  658 False Negative: 60
Tweets:  10
Cluster Label: 138 9
472  472  472  472  472  472  472  472  472  
Event 63 :
Cluster_noise:  1 False Negative: 0
Tweets:  715
Cluster Label: 139 8
2  2  2  2  2  2  2  2  
Cluster Label: 140 10
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 141 8
282  282  477  477  477  282  282  477  
Cluster Label: 142 5
472  472  472  472  472  
Cluster Label: 143 5
42

Event 99 :
Cluster_noise:  705 False Negative: 34
Tweets:  108
Cluster Label: 196 98
4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  


In [177]:
hdbscan_show = create_clusters_show(df,2,0.5,1,0,'hdbscan')

Event 0 :
Cluster_noise:  1442 False Negative: 362
Tweets:  802
Cluster Label: 0 20
394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  501  501  
Cluster Label: 1 6
487  487  487  487  394  394  
Cluster Label: 2 6
394  394  -1  394  394  -1  
Cluster Label: 3 12
501  501  487  487  394  394  394  394  487  487  394  394  
Cluster Label: 4 5
-1  -1  19  19  -1  
Cluster Label: 5 16
487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  
Cluster Label: 6 14
-1  487  487  487  487  487  487  487  487  487  487  487  487  -1  
Cluster Label: 7 61
487  487  -1  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  
Cluster Label: 8 8
-1  19  19  -1  394  394  394  394  
Cluster Label: 9 6
-1  -1

18  18  18  18  18  
Cluster Label: 128 16
18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  
Cluster Label: 129 8
18  18  18  18  18  18  18  18  
Cluster Label: 130 6
18  18  18  18  18  18  
Cluster Label: 131 20
18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  
Cluster Label: 132 20
18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  
Cluster Label: 133 20
18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  
Cluster Label: 134 13
18  18  18  18  18  18  18  18  18  18  18  18  18  
Event 3 :
Cluster_noise:  842 False Negative: 28
Tweets:  13
Event 4 :
Cluster_noise:  614 False Negative: 34
Tweets:  12
Event 5 :
Cluster_noise:  1 False Negative: 0
Tweets:  447
Cluster Label: 135 414
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  382  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  245  -1  -1  -1  -1  -1  -1  -1  -1  -1

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 161 11
246  246  246  246  246  246  -1  246  246  246  246  
Cluster Label: 162 13
239  239  239  380  380  239  239  380  238  238  238  380  380  
Cluster Label: 163 9
239  380  238  238  239  380  380  239  238  
Cluster Label: 164 12
239  426  426  239  426  426  239  426  426  239  239  426  
Cluster Label: 165 10
239  239  239  426  426  239  239  239  426  426  
Event 19 :
Cluster_noise:  663 False Negative: 31
Tweets:  51
Cluster Label: 166 13
246  246  246  246  246  246  246  246  246  246  246  246  246  
Cluster Label: 167 16
239  239  426  426  239  426  426  239  239  239  239  239  239  239  426  426  
Cluster Label: 168 11
239  380  426  426  380  239  239  426  380  426  239  
Event 20 :
Cluster_noise:  754 False Negative: 41
Tweets:  64
Cluster Label: 169 10
75  75  75  75  75  75  75  75  75  75  
Cluster Label: 170 15
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -

Event 29 :
Cluster_noise:  545 False Negative: 51
Tweets:  14
Event 30 :
Cluster_noise:  408 False Negative: 22
Tweets:  10
Event 31 :
Cluster_noise:  325 False Negative: 23
Tweets:  27
Cluster Label: 183 5
239  239  239  239  239  
Cluster Label: 184 8
239  239  239  239  239  239  239  239  
Event 32 :
Cluster_noise:  289 False Negative: 26
Tweets:  10
Event 33 :
Cluster_noise:  285 False Negative: 35
Tweets:  29
Cluster Label: 185 16
422  422  422  422  422  422  422  422  422  422  422  422  422  422  422  422  
Cluster Label: 186 11
239  426  239  239  239  239  426  426  239  239  239  
Event 34 :
Cluster_noise:  376 False Negative: 34
Tweets:  84
Cluster Label: 187 56
106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  
Cluster Label: 188 10
239  239  239

1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
Cluster Label: 291 8
1  1  1  1  1  1  1  1  
Cluster Label: 292 13
1  1  1  1  1  1  1  1  1  1  1  1  1  
Cluster Label: 293 10
-1  1  1  1  1  1  1  1  1  1  
Cluster Label: 294 19
1  1  1  1  1  1  1  -1  1  1  1  1  1  1  1  1  1  1  1  
Cluster Label: 295 6
85  1  1  1  85  1  
Cluster Label: 296 5
1  1  1  1  1  
Cluster Label: 297 39
1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  -1  1  1  1  1  -1  1  1  1  1  1  1  1  1  1  1  -1  1  1  
Cluster Label: 298 11
1  1  1  1  -1  1  1  1  1  1  1  
Cluster Label: 299 10
85  85  1  85  1  85  1  1  1  1  
Cluster Label: 300 9
1  1  1  1  1  1  1  1  1  
Cluster Label: 301 17
1  1  1  1  1  1  1  1  1  1  1  1  85  85  85  85  1  
Cluster Label: 302 8
1  1  -1  1  1  1  1  1  
Cluster Label: 303 9
1  1  -1  -1  1  1  1  1  1  
Cluster Label: 30

...

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  423  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  426  -1  -1  -1  -1  -1  -1  -1  -1  -1  421  -1  -1  426  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  88  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  0  -1  -1  -1  -1  -1  -1  423  -1  426  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  424  -1  424  -1  424  421  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  420  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  427  88  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  89  -1  -1  -1  -1  -1  -1  -1  427  -1  -1  -1  -1  -1  -1  0  -1  -1  -1  -1  -1  -1  -1  -1  -1  89  -1  -1  -1  -1  -1  -1  429  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  

Event 64 :
Cluster_noise:  660 False Negative: 45
Tweets:  19
Cluster Label: 598 12
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 599 5
-1  -1  -1  -1  -1  
Event 65 :
Cluster_noise:  690 False Negative: 61
Tweets:  10
Event 66 :
Cluster_noise:  711 False Negative: 71
Tweets:  16
Cluster Label: 600 8
477  282  282  477  477  477  282  282  
Cluster Label: 601 8
282  477  282  477  282  477  477  282  
Event 67 :
Cluster_noise:  808 False Negative: 156
Tweets:  47
Cluster Label: 602 10
477  282  282  477  477  -1  477  477  477  477  
Cluster Label: 603 37
91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  -1  91  -1  91  -1  -1  -1  91  91  91  91  
Event 68 :
Cluster_noise:  788 False Negative: 150
Tweets:  147
Cluster Label: 604 12
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 605 30
86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  94  -1  -1  -1  -1  -1  -1  -1  280  280  -1  -1  -1  280  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  3  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  3  -1  3  -1  -1  3  -1  -1  -1  -1  3  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  279  279  -1  -1  280  -1  -1  -1  -1  3  -1  3  -1  -1  -1  -1  -1  -1  -1  280  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  280  3  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -

Cluster Label: 652 22
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 653 203
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  474  -1  -1  -1  -1  -1  474  -1  -1  -1  275  -1  -1  -1  -1  -1  -1  283  -1  -1  -1  472  472  -1  -1  -1  -1  -1  -1  472  -1  -1  472  -1  -1  -1  280  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  478  -1  -1  -1  -1  -1  275  280  280  -1  -1  -1  -1  -1  -1  -1  -1  -1  285  -1  -1  275  -1  -1  -1  -1  -1  472  -1  -1  -1  -1  474  -1  -1  -1  -1  -1  478  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  275  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  474  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  474  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  275  -1  -1  -1  -1  275  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 654 5
-1  -1  2

Event 95 :
Cluster_noise:  709 False Negative: 67
Tweets:  20
Event 96 :
Cluster_noise:  762 False Negative: 140
Tweets:  39
Event 97 :
Cluster_noise:  674 False Negative: 21
Tweets:  32
Cluster Label: 665 6
71  71  71  71  71  71  
Cluster Label: 666 16
71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  
Event 98 :
Cluster_noise:  729 False Negative: 16
Tweets:  32
Cluster Label: 667 22
71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  71  
Cluster Label: 668 8
71  71  71  71  71  71  71  71  
Event 99 :
Cluster_noise:  705 False Negative: 34
Tweets:  108
Cluster Label: 669 5
4  4  4  4  4  
Cluster Label: 670 7
4  4  4  4  4  4  4  
Cluster Label: 671 5
4  4  4  4  4  
Cluster Label: 672 5
4  4  4  4  4  
Cluster Label: 673 26
4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  
Cluster Label: 674 19
4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  
Cluster Label: 675 6
4  4  4  4  4  4  


In [178]:
hierarchical = create_clusters_show(df,2,0.5,1,0,'hierarchical')

Event 0 :
Cluster_noise:  1442 False Negative: 362
Tweets:  802
Cluster Label: 0 121
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  394  394  394  394  394  394  501  501  394  394  394  394  394  394  394  394  394  394  501  501  501  501  394  394  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  501  -1  501  501  501  501  501  501  501  501  501  501  -1  501  501  501  501  501  501  -1  501  501  501  501  -1  501  501  394  394  -1  -1  -1  501  501  -1  -1  -1  487  487  487  487  501  501  394  394  -1  -1  -1  -1  394  394  -1  
Cluster Label: 1 207
19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  -1  19  19  19  19  -1  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  19  1

504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  -1  504  504  504  504  504  504  -1  504  504  504  504  504  504  -1  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  -1  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  -1  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  -1  504  504  504  504  504  504  -1  -1  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  504  -1  -1  -1  504  504  504  5

383  383  383  383  383  383  383  383  383  
Cluster Label: 43 4
75  75  75  75  
Cluster Label: 44 2
241  241  
Event 9 :
Cluster_noise:  307 False Negative: 28
Tweets:  11
Cluster Label: 45 6
243  243  243  243  243  243  
Cluster Label: 46 1
243  
Cluster Label: 47 2
243  243  
Cluster Label: 48 1
243  
Cluster Label: 49 1
243  
Event 10 :
Cluster_noise:  379 False Negative: 44
Tweets:  45
Cluster Label: 50 8
243  243  243  243  243  243  243  243  
Cluster Label: 51 13
243  243  243  243  243  243  243  243  243  243  243  243  243  
Cluster Label: 52 14
243  243  243  243  243  243  243  243  243  243  243  243  243  243  
Cluster Label: 53 6
243  243  243  243  243  243  
Cluster Label: 54 4
243  243  243  243  
Event 11 :
Cluster_noise:  451 False Negative: 37
Tweets:  51
Cluster Label: 55 15
241  241  241  241  241  241  241  241  241  241  241  241  241  241  241  
Cluster Label: 56 6
383  383  383  383  383  383  
Cluster Label: 57 23
386  243  243  243  243  243  243  243  

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  246  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  243  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

Event 34 :
Cluster_noise:  376 False Negative: 34
Tweets:  84
Cluster Label: 170 36
106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  
Cluster Label: 171 6
239  239  239  239  239  239  
Cluster Label: 172 18
419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  419  
Cluster Label: 173 20
106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  106  
Cluster Label: 174 4
426  239  426  239  
Event 35 :
Cluster_noise:  484 False Negative: 47
Tweets:  159
Cluster Label: 175 78
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
Cluster Label: 176 46
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

...

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  89  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 274 3
426  426  426  
Event 55 :
Cluster_noise:  370 False Negative: 58
Tweets:  45
Cluster Label: 275 15
148  148  148  148  148  148  148  148  148  148  148  148  148  148  148  
Cluster Label: 276 6
89  -1  89  89  89  89  
Cluster Label: 277 12
0  0  2  0  2  2  2  2  0  0  0  0  
Cluster Label: 278 8
89  280  280  89  89  280  89  280  
Cluster Label: 279 4
280  89  280  89  
Event 56 :
Cluster_noise:  282 False Negative: 33
Tweets:  73
Cluster Label: 280 25
2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  0  2  2  2  -1  -1  0  
Cluster Label: 281 17
89  89  280  280  89  280  280  89  89  280  89  89  280  89  89  280  280  
Cluster Label: 282 9
89  89  89  89  89  89  280  89  280  
Cluster Label: 283 14
89  280  280  89  89  89  89  280  280  280  280  89  89  89  
Cluster Label: 284 8
280  89  280  89  280  280  89  89  
Event 57 :
Cluster_noise:  301 

-1  
Event 66 :
Cluster_noise:  711 False Negative: 71
Tweets:  16
Cluster Label: 330 3
282  477  282  
Cluster Label: 331 4
477  477  282  282  
Cluster Label: 332 4
282  477  477  282  
Cluster Label: 333 4
477  282  282  477  
Cluster Label: 334 1
477  
Event 67 :
Cluster_noise:  808 False Negative: 156
Tweets:  47
Cluster Label: 335 16
91  91  91  91  91  91  -1  91  -1  -1  -1  -1  91  91  91  91  
Cluster Label: 336 5
282  282  477  477  -1  
Cluster Label: 337 19
91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  91  
Cluster Label: 338 5
477  477  477  477  477  
Cluster Label: 339 2
91  91  
Event 68 :
Cluster_noise:  788 False Negative: 150
Tweets:  147
Cluster Label: 340 32
284  284  480  480  284  480  480  480  284  284  480  284  480  284  480  284  284  480  480  284  284  480  284  284  480  480  480  480  284  284  480  480  
Cluster Label: 341 30
86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86  86

Event 75 :
Cluster_noise:  0 False Negative: 0
Tweets:  630
Cluster Label: 375 66
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  
Cluster Label: 376 415
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  3  -1  3  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1 

Event 81 :
Cluster_noise:  1 False Negative: 0
Tweets:  275
Cluster Label: 405 161
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 406 89
472  -1  275  275  465  -1  275  -1  -1  -1  -1  -1  2  -1  478  279  -1  -1  -1  -1  280  -1  275  -1  -1  -1  -1  278  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  479  279  -1  464  -1  -1  -1  -1  -1  -1  -1  47

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1

In [179]:
kmeans = create_clusters_show(df,2,0.5,1,0,'kmeans')

Event 0 :
Cluster_noise:  1442 False Negative: 362
Tweets:  802
Cluster Label: 0 146
487  487  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  487  487  -1  487  487  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  -1  -1  19  19  -1  -1  -1  -1  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  -1  487  487  487  487  487  487  487  487  487  487  487  487  -1  487  487  487  487  487  487  -1  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  -1  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  -1  487  487  487  487  -1  -1  487  487  -1  -1  -1  
Cluster Label: 1 321
394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  -1  394  394  3

18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  -1  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  -1  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  

C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


Cluster Label: 23 1
-1  
Cluster Label: 24 19
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 25 1
-1  
Cluster Label: 26 114
-1  382  -1  -1  -1  -1  245  -1  -1  -1  -1  -1  75  -1  -1  -1  -1  241  -1  -1  -1  -1  383  -1  -1  -1  -1  -1  -1  240  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  241  383  -1  384  -1  384  -1  -1  -1  386  -1  -1  75  75  -1  -1  -1  -1  -1  420  -1  -1  -1  -1  -1  -1  241  241  -1  -1  -1  -1  -1  384  384  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  75  75  383  -1  -1  -1  -1  -1  -1  241  -1  -1  238  380  238  380  -1  383  383  -1  -1  -1  -1  -1  -1  383  -1  
Cluster Label: 27 312
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  75  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  75  -1  -1  75  -1  -1  -1  -1  -1  -1  -1  -1 

Event 18 :
Cluster_noise:  750 False Negative: 51
Tweets:  96
Cluster Label: 88 19
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 89 42
239  239  239  239  239  426  426  239  239  239  426  239  426  239  239  380  380  239  239  380  238  238  238  380  239  239  380  239  426  426  239  426  426  239  239  239  239  426  239  426  239  426  
Cluster Label: 90 13
239  380  238  238  239  380  380  239  238  239  239  426  426  
Cluster Label: 91 13
246  246  246  246  246  246  -1  246  246  246  246  246  246  
Cluster Label: 92 9
239  239  426  426  239  239  239  426  426  
Event 19 :
Cluster_noise:  663 False Negative: 31
Tweets:  51
Cluster Label: 93 12
246  246  246  246  246  246  246  246  246  -1  246  246  
Cluster Label: 94 17
239  239  380  426  426  380  239  239  426  380  239  239  239  239  239  426  239  
Cluster Label: 95 14
239  239  239  426  426  239  426  426  239  239  239  239  239  239  
Cluster Label: 96 6
239  23

...

Cluster Label: 405 1
-1  
Cluster Label: 406 156
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  282  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  285  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  275  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 407 104
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  474  -1  -1  -1  -1  474  -1  -1  275  -1  -1  -1  -1  -1  283  -1  -1  -1  -1  -1  -1  -1  280  -1  -1  -1  -1  -1  478  -1  275  280  280  -1  -1  -1  -1  275  -1  472  -1  -1  -1  -1  474  -1  -1  -1  478  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 

C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


Cluster Label: 430 4
284  284  480  480  
Cluster Label: 431 4
480  284  480  284  
Cluster Label: 432 4
284  480  284  480  
Cluster Label: 433 4
480  284  284  480  
Event 88 :
Cluster_noise:  1 False Negative: 0
Tweets:  739
Cluster Label: 434 2
282  282  
Cluster Label: 435 537
4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  285  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -

C:\Users\giann\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


Event 90 :
Cluster_noise:  707 False Negative: 31
Tweets:  10
Cluster Label: 443 5
278  278  278  278  278  
Cluster Label: 444 1
278  
Cluster Label: 445 1
278  
Cluster Label: 446 2
278  278  
Cluster Label: 447 1
278  
Event 91 :
Cluster_noise:  716 False Negative: 33
Tweets:  13
Cluster Label: 448 2
280  280  
Cluster Label: 449 4
280  280  280  280  
Cluster Label: 450 3
280  280  280  
Cluster Label: 451 3
280  280  280  
Cluster Label: 452 1
280  
Event 92 :
Cluster_noise:  1 False Negative: 0
Tweets:  694
Cluster Label: 453 1
-1  
Cluster Label: 454 523
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1

In [180]:
affinity = create_clusters_show(df,2,0.5,1,0,'affinity')

Event 0 :
Cluster_noise:  1442 False Negative: 362
Tweets:  802
Cluster Label: 0 29
394  394  394  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  
Cluster Label: 1 18
394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  
Cluster Label: 2 2
394  394  
Cluster Label: 3 9
-1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 4 1
-1  
Cluster Label: 5 23
487  487  -1  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  487  
Cluster Label: 6 66
394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  394  -1  394  394  394  394  394  394  394  394  394  394  394  394  394  
Cluster Label: 7 11
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  


504  504  504  504  504  504  504  504  -1  
Cluster Label: 162 2
504  504  
Cluster Label: 163 2
504  504  
Cluster Label: 164 2
504  504  
Cluster Label: 165 2
504  504  
Cluster Label: 166 2
502  502  
Cluster Label: 167 2
502  502  
Cluster Label: 168 6
504  504  504  504  504  504  
Cluster Label: 169 2
504  504  
Cluster Label: 170 1
-1  
Cluster Label: 171 18
18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  
Cluster Label: 172 10
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 173 1
-1  
Cluster Label: 174 2
504  504  
Cluster Label: 175 4
504  504  504  504  
Cluster Label: 176 10
504  504  504  504  504  504  504  504  504  504  
Cluster Label: 177 6
18  18  18  18  18  18  
Cluster Label: 178 1
-1  
Cluster Label: 179 2
75  75  
Cluster Label: 180 2
18  18  
Cluster Label: 181 20
18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  18  
Cluster Label: 182 2
18  18  
Cluster Label: 183 2
504  504  
Cluster Label: 184 2
18  

18  18  
Cluster Label: 320 1
75  
Cluster Label: 321 2
18  18  
Cluster Label: 322 1
-1  
Cluster Label: 323 2
18  18  
Cluster Label: 324 2
18  18  
Cluster Label: 325 2
75  75  
Cluster Label: 326 2
18  18  
Cluster Label: 327 2
18  18  
Cluster Label: 328 1
75  
Cluster Label: 329 1
75  
Cluster Label: 330 2
18  18  
Cluster Label: 331 2
18  18  
Cluster Label: 332 2
18  18  
Cluster Label: 333 1
75  
Cluster Label: 334 1
75  
Cluster Label: 335 2
75  75  
Cluster Label: 336 2
18  18  
Cluster Label: 337 1
75  
Cluster Label: 338 1
75  
Cluster Label: 339 2
18  18  
Cluster Label: 340 2
18  18  
Cluster Label: 341 7
18  18  18  18  18  18  18  
Cluster Label: 342 2
18  18  
Cluster Label: 343 2
18  18  
Cluster Label: 344 2
18  18  
Cluster Label: 345 2
75  75  
Cluster Label: 346 1
75  
Cluster Label: 347 1
75  
Cluster Label: 348 1
75  
Cluster Label: 349 4
18  18  18  18  
Cluster Label: 350 1
75  
Cluster Label: 351 1
75  
Cluster Label: 352 2
18  18  
Cluster Label: 353 2
18  

Event 10 :
Cluster_noise:  379 False Negative: 44
Tweets:  45
Cluster Label: 479 10
243  243  243  243  243  243  243  243  243  243  
Cluster Label: 480 1
243  
Cluster Label: 481 10
243  243  243  243  243  243  243  243  243  243  
Cluster Label: 482 11
243  243  243  243  243  243  243  243  243  243  243  
Cluster Label: 483 5
243  243  243  243  243  
Cluster Label: 484 8
243  243  243  243  243  243  243  243  
Event 11 :
Cluster_noise:  451 False Negative: 37
Tweets:  51
Cluster Label: 485 4
383  383  383  383  
Cluster Label: 486 7
243  243  243  243  243  243  243  
Cluster Label: 487 2
383  383  
Cluster Label: 488 4
383  383  383  383  
Cluster Label: 489 16
386  243  243  243  243  243  243  243  243  243  243  243  243  243  243  243  
Cluster Label: 490 3
243  243  243  
Cluster Label: 491 15
241  241  241  241  241  241  241  241  241  241  241  241  241  241  241  
Event 12 :
Cluster_noise:  525 False Negative: 36
Tweets:  97
Cluster Label: 492 14
243  243  243  243  2

-1  
Cluster Label: 636 1
-1  
Cluster Label: 637 1
-1  
Cluster Label: 638 3
246  246  246  
Cluster Label: 639 1
-1  
Cluster Label: 640 29
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 641 1
-1  
Cluster Label: 642 1
-1  
Cluster Label: 643 1
-1  
Cluster Label: 644 1
-1  
Cluster Label: 645 1
-1  
Cluster Label: 646 1
-1  
Cluster Label: 647 2
-1  -1  
Cluster Label: 648 1
-1  
Cluster Label: 649 48
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  240  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 650 1
-1  
Cluster Label: 651 1
-1  
Cluster Label: 652 1
-1  
Cluster Label: 653 1
-1  
Cluster Label: 654 1
-1  
Cluster Label: 655 32
-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 656 1
-1  
Cluste

...

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  3  -1  -1  -1  -1  -1  -1  -1  -1  
Cluster Label: 2165 1
-1  
Cluster Label: 2166 1
-1  
Cluster Label: 2167 1
-1  
Cluster Label: 2168 1
-1  
Cluster Label: 2169 162
-1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  276  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  3  -1  3  -1  -1  3  -1  -1  -1  3  -1  -1  -1  -1  -1  -1  3  -1  -1  3  -1  -1  -1  -1  3  -1  -1  3  -1  -1  -1  -1  3  -1  3  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  3  3  -1  -1  -1  
Cluster Label: 2170 1
-1  
Cluster Label: 2171 1
-1  
Cluster Label: 2172 14
-1  3  3  3  3  3  3  3  3  3  3  3  -1  -1  


Event 76 :
Cluster_noise:  1014 False Negative: 416
Tweets:  288
Cluster Label: 2285 11
3  3  3  3  3  3  3  3  3  3  3  
Cluster Label: 2286 2
3  3  
Cluster Label: 2287 2
3  3  
Cluster Label: 2288 41
3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  -1  -1  3  3  
Cluster Label: 2289 6
3  3  3  3  3  3  
Cluster Label: 2290 1
3  
Cluster Label: 2291 2
505  505  
Cluster Label: 2292 11
3  3  3  3  3  3  3  3  3  3  3  
Cluster Label: 2293 3
3  3  3  
Cluster Label: 2294 2
3  3  
Cluster Label: 2295 1
-1  
Cluster Label: 2296 2
3  3  
Cluster Label: 2297 2
505  505  
Cluster Label: 2298 3
3  3  3  
Cluster Label: 2299 6
3  -1  94  3  3  94  
Cluster Label: 2300 2
505  505  
Cluster Label: 2301 17
3  -1  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  
Cluster Label: 2302 6
3  3  3  3  3  3  
Cluster Label: 2303 2
3  3  
Cluster Label: 2304 2
3  3  
Cluster Label: 2305 2
3  3  
Cluster Label: 2306 2
3  3  
Cluster Label: 2307 2


Cluster Label: 2430 12
275  275  275  275  275  275  275  275  275  -1  275  275  
Cluster Label: 2431 4
95  475  95  475  
Cluster Label: 2432 4
95  475  475  95  
Cluster Label: 2433 7
275  275  275  275  275  275  275  
Cluster Label: 2434 1
475  
Cluster Label: 2435 4
95  95  475  475  
Cluster Label: 2436 3
275  275  275  
Cluster Label: 2437 4
95  475  475  95  
Cluster Label: 2438 1
475  
Cluster Label: 2439 1
475  
Cluster Label: 2440 1
95  
Cluster Label: 2441 1
95  
Cluster Label: 2442 1
95  
Cluster Label: 2443 1
475  
Cluster Label: 2444 1
475  
Cluster Label: 2445 1
95  
Cluster Label: 2446 1
475  
Cluster Label: 2447 2
95  475  
Cluster Label: 2448 1
95  
Cluster Label: 2449 1
95  
Cluster Label: 2450 1
475  
Cluster Label: 2451 1
475  
Cluster Label: 2452 1
95  
Cluster Label: 2453 1
475  
Cluster Label: 2454 1
95  
Cluster Label: 2455 1
95  
Cluster Label: 2456 1
475  
Cluster Label: 2457 1
475  
Cluster Label: 2458 1
475  
Cluster Label: 2459 1
95  
Cluster Label: 2460

-1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  480  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  285  -1  -1  -1  -1  285  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  282  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  4  -1  -1  -1  
Cluster Label: 2563 1
-1  
Cluster Label: 2564 1
-1  
Cluster Label: 2565 42
-1  -1  -1  -1 

Event 95 :
Cluster_noise:  709 False Negative: 67
Tweets:  20
Cluster Label: 2690 1
71  
Cluster Label: 2691 8
71  71  71  71  71  71  71  71  
Cluster Label: 2692 5
71  71  71  71  71  
Cluster Label: 2693 2
71  71  
Cluster Label: 2694 2
71  71  
Cluster Label: 2695 2
71  71  
Event 96 :
Cluster_noise:  762 False Negative: 140
Tweets:  39
Cluster Label: 2696 1
-1  
Cluster Label: 2697 1
4  
Cluster Label: 2698 13
4  4  4  -1  4  4  4  4  4  4  4  4  4  
Cluster Label: 2699 1
4  
Cluster Label: 2700 11
4  4  4  4  4  4  4  4  -1  4  4  
Cluster Label: 2701 1
4  
Cluster Label: 2702 9
4  4  4  4  4  4  4  4  4  
Cluster Label: 2703 1
4  
Cluster Label: 2704 1
4  
Event 97 :
Cluster_noise:  674 False Negative: 21
Tweets:  32
Cluster Label: 2705 4
71  71  71  71  
Cluster Label: 2706 4
71  71  71  71  
Cluster Label: 2707 4
71  71  71  71  
Cluster Label: 2708 2
71  71  
Cluster Label: 2709 1
71  
Cluster Label: 2710 1
71  
Cluster Label: 2711 1
71  
Cluster Label: 2712 1
71  
Cluster La

In [29]:
# ngram = [2]
# epsilon = [0.4,0.5,0.6,0.7,0.8,0.9]
# mindf = [1]
# maxdf = [0]
# distance_metric = ['cosine']
# dbscan_json = []
# final_count = 0
# for w in range(len(distance_metric)):
#     for i in range(len(ngram)):
#         for j in range(len(epsilon)):
#             for k in range(len(mindf)):
#                 for l in range(len(maxdf)):
#                     temp = create_clusters(df,ngram[i],epsilon[j],mindf[k],maxdf[l],0,distance_metric[w])
#                     if temp: 
#                         dbscan_json.append(temp)
#                         print("Done",final_count)
#                     final_count+=1

# root_object = {"root":dbscan_json}
# with open('dbscan_parameters_evaluation.json', 'w') as f:
#     json.dump(root_object, f)

In [30]:
# epsilon = [0.5]
# hdbscan_json = []
# final_count = 0 
# for w in range(len(distance_metric)):
#     for i in range(len(ngram)):
#         for j in range(len(epsilon)):
#             for k in range(len(mindf)):
#                 for l in range(len(maxdf)):
#                     temp = create_clusters(df,ngram[i],epsilon[j],mindf[k],maxdf[l],1,distance_metric[w])
#                     if temp:
#                         hdbscan_json.append(temp)
#                         print("Done",final_count)
#                     final_count+=1

# root_object = {"root":hdbscan_json}
# with open('hdbscan_parameters_evaluation.json', 'w') as f:
#     json.dump(root_object, f)